# Scraping da https://www.alimentinutrizione.it/tabelle-nutrizionali/ricerca-per-categoria senza file excel

In [ ]:
import requests
from bs4 import BeautifulSoup

# URL della pagina iniziale delle categorie di alimenti
base_url = "https://www.alimentinutrizione.it/tabelle-nutrizionali/ricerca-per-categoria"

# Funzione per ottenere il contenuto HTML di una pagina
def get_html(url):
    response = requests.get(url)
    if response.status_code == 200:
        return response.text
    else:
        print(f"Errore {response.status_code} nella richiesta a {url}")
        return None

# Funzione per estrarre i link delle ricette
def get_recipe_links(html):
    soup = BeautifulSoup(html, 'html.parser')
    recipe_links = []
    for link in soup.find_all('a', href=True):
        href = link.get('href')
        if href.startswith('/tabelle-nutrizionali/'):
            recipe_links.append(href)
    return recipe_links[:3]  # Limita ai primi 3 risultati

# Funzione per estrarre i dati nutrizionali di un alimento
def extract_food_data(html):
    soup = BeautifulSoup(html, 'html.parser')
    food_data = []

    # Modifica il selettore in base alla struttura della pagina
    table = soup.find('table')
    headers = []
    if table:
        headers = [header.text.strip() for header in table.find_all('th')]
        rows = table.find_all('tr')
        for row in rows[1:]:
            cells = row.find_all('td')
            food_data.append([cell.text.strip() for cell in cells])
    else:
        print("Nessuna tabella trovata.")

    return headers, food_data

# Ottieni i link delle categorie di alimenti
initial_html = get_html(base_url)
category_links = get_recipe_links(initial_html)

# Itera su ogni link di categoria per estrarre i dati nutrizionali degli alimenti
for link in category_links:
    category_url = f"https://www.alimentinutrizione.it{link}"
    category_html = get_html(category_url)
    if category_html:
        category_headers, food_data = extract_food_data(category_html)

        # Stampa headers e food_data per ciascuna categoria
        print(f"Headers per {link}: {category_headers}")
        print(f"Dati per {link}: {food_data}")

Headers per /tabelle-nutrizionali/120010: ['Acciuga o alice']
Dati per /tabelle-nutrizionali/120010: [['Categoria', 'Prodotti della pesca'], ['Codice Alimento', '120010'], ['Nome Scientifico', 'Engraulis enchrasicholus'], ['English Name', 'Anchovy, fresh, raw'], ['Parte Edibile', '75 %'], ['Porzione', '150 g']]
Headers per /tabelle-nutrizionali/120020: ["Acciuga o alice, sott'olio"]
Dati per /tabelle-nutrizionali/120020: [['Categoria', 'Prodotti della pesca'], ['Codice Alimento', '120020'], ['Nome Scientifico', 'Engraulis enchrasicholus'], ['English Name', 'Anchovy, in oil'], ['Parte Edibile', '100 %'], ['Porzione', '50 g']]
Headers per /tabelle-nutrizionali/120030: ['Acciuga o alice, sotto sale']
Dati per /tabelle-nutrizionali/120030: [['Categoria', 'Prodotti della pesca'], ['Codice Alimento', '120030'], ['Nome Scientifico', 'Engraulis enchrasicholus'], ['English Name', 'Anchovy, salted'], ['Parte Edibile', '50 %'], ['Porzione', '50 g']]
Headers per /tabelle-nutrizionali/005000: ['Agl

# Scraping da https://www.alimentinutrizione.it/tabelle-nutrizionali/ricerca-per-categoria con file excel, ma senza dettagli nutrizionali ...

In [10]:
import requests
from bs4 import BeautifulSoup
import openpyxl
import logging

logging.basicConfig(level=logging.INFO)  # Imposta il livello di logging

BASE_URL = "https://www.alimentinutrizione.it/tabelle-nutrizionali/ricerca-per-ordine-alfabetico"

def get_html(url):
    """
    Ottiene il contenuto HTML di una pagina.

    :param url: URL della pagina da cui ottenere l'HTML
    :return: Contenuto HTML della pagina, o None se si verifica un errore
    """
    try:
        response = requests.get(url)
        response.raise_for_status()  # Genera un'eccezione per errori HTTP
        return response.text
    except requests.exceptions.RequestException as e:
        logging.error(f"Errore nella richiesta a {url}: {e}")
        return None

def get_recipe_links(html):
    """
    Estrae i link delle ricette dalla pagina iniziale delle categorie di alimenti.

    :param html: Contenuto HTML della pagina
    :return: Lista di link delle ricette (limitato ai primi 3 risultati)
    """
    soup = BeautifulSoup(html, 'html.parser')
    recipe_links = []
    for link in soup.find_all('a', href=True):
        href = link.get('href')
        if href.startswith('/tabelle-nutrizionali/'):
            recipe_links.append(href)
    return recipe_links[:3]

def extract_food_data(html):
    """
    Estrae i dati nutrizionali di un alimento dalla pagina.

    :param html: Contenuto HTML della pagina
    :return: Tuple contenente gli headers e i dati nutrizionali
    """
    soup = BeautifulSoup(html, 'html.parser')
    food_data = []

    table = soup.find('table')
    headers = []
    if table:
        headers = [header.text.strip() for header in table.find_all('th')]
        rows = table.find_all('tr')
        for row in rows[1:]:
            cells = row.find_all('td')
            food_data.append([cell.text.strip() for cell in cells])
    else:
        logging.warning("Nessuna tabella trovata.")
    print(food_data)
    return headers, food_data

def main():
    workbook = openpyxl.Workbook()
    sheet = workbook.active
    sheet.title = "Dati Nutrizionali"
    sheet.append(["Link", "Nome Alimento", "Categoria", "Codice Alimento", "Nome Scientifico", "English Name", "Parte Edibile", "Porzione"])

    initial_html = get_html(BASE_URL)
    if initial_html:
        category_links = get_recipe_links(initial_html)

        for link in category_links:
            category_url = f"https://www.alimentinutrizione.it{link}"
            category_html = get_html(category_url)
            if category_html:
                category_headers, food_data = extract_food_data(category_html)
                logging.info(f"Headers per {link}: {category_headers}")
                logging.info(f"Dati per {link}: {food_data}")

                if food_data:
                    full_link = f"https://www.alimentinutrizione.it{link}"  # Link completo
                    row_data = [full_link, category_headers[0]]  # Link e Nome Alimento
                    row_data.extend([item[1] for item in food_data])  # Aggiungi gli altri dati
                    sheet.append(row_data)

        workbook.save("dati_nutrizionali.xlsx")
        logging.info("Dati salvati in 'dati_nutrizionali.xlsx'")
    else:
        logging.error("Impossibile ottenere il contenuto HTML dalla pagina iniziale.")

if __name__ == "__main__":
    main()

[['Categoria', 'Prodotti della pesca'], ['Codice Alimento', '120010'], ['Nome Scientifico', 'Engraulis enchrasicholus'], ['English Name', 'Anchovy, fresh, raw'], ['Parte Edibile', '75 %'], ['Porzione', '150 g']]
[['Categoria', 'Prodotti della pesca'], ['Codice Alimento', '120020'], ['Nome Scientifico', 'Engraulis enchrasicholus'], ['English Name', 'Anchovy, in oil'], ['Parte Edibile', '100 %'], ['Porzione', '50 g']]
[['Categoria', 'Prodotti della pesca'], ['Codice Alimento', '120030'], ['Nome Scientifico', 'Engraulis enchrasicholus'], ['English Name', 'Anchovy, salted'], ['Parte Edibile', '50 %'], ['Porzione', '50 g']]


# ESTRAZIONE DEI VALORI DI DETTAGLIO DI OGNI ALIMENTO

In [ ]:
import requests
from bs4 import BeautifulSoup

# URL della pagina web da analizzare
url = "https://www.alimentinutrizione.it/tabelle-nutrizionali/120010"

# Recupero della pagina web
response = requests.get(url)
response.raise_for_status()  # Verifica che la richiesta sia andata a buon fine

# Parsing della pagina web con BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

# Lista degli elementi da estrarre
elements_to_extract = {
    "Acqua (g)": "g",
    "Energia (kcal)": "kcal",
    "Energia (kJ)": "kJ",
    "Proteine (g)": "g (N x 6,25)",
    "Colesterolo (mg)": "mg",
    "Carboidrati disponibili (g)": "g",
    "Amido (g)": "g",
    "Zuccheri solubili (g)": "g",
    "Alcool (g)": "g",
    "Fibra totale (g)": "g"
}

# Dizionario per memorizzare i risultati
results = {}

# Estrazione dei dati
for row in soup.find_all("tr", class_="corponutriente"):
    cells = row.find_all("td")
    if len(cells) >= 3:
        element_name = cells[0].get_text(strip=True)
        if element_name in elements_to_extract:
            element_value = cells[2].get_text(strip=True)
            unit = elements_to_extract[element_name]
            results[element_name] = f"{element_value} {unit}"

# Stampa i risultati
for element, value in results.items():
    print(f"{element}: {value}")


Acqua (g): 76.5 g
Energia (kcal): 96 kcal
Energia (kJ): 403 kJ
Proteine (g): 16.8 g (N x 6,25)
Colesterolo (mg): 61 mg
Carboidrati disponibili (g): 1.5 g
Amido (g): 0 g
Zuccheri solubili (g): 1.5 g
Alcool (g): 0 g
Fibra totale (g): 0 g


# LINK CREATI CORRETAMENTE E DETTAGLI ALIMENTO SALVATI CORRETTAMENTE - MANCANO NOME ALIMENTO...

In [2]:
import requests
from bs4 import BeautifulSoup
import openpyxl
import logging

logging.basicConfig(level=logging.INFO)

BASE_URL = "https://www.alimentinutrizione.it/tabelle-nutrizionali/ricerca-per-ordine-alfabetico"

def get_html(url):
    try:
        response = requests.get(url)
        response.raise_for_status()
        return response.text
    except requests.exceptions.RequestException as e:
        logging.error(f"Errore nella richiesta a {url}: {e}")
        return None

def get_recipe_links(html):
    soup = BeautifulSoup(html, 'html.parser')
    recipe_links = []
    for link in soup.find_all('a', href=True):
        href = link.get('href')
        if href.startswith('/tabelle-nutrizionali/'):
            recipe_links.append(href)
    return recipe_links[:3]

def extract_food_data(html):
    soup = BeautifulSoup(html, 'html.parser')

    elements_to_extract = {
        "Acqua (g)": "g",
        "Energia (kcal)": "kcal",
        "Energia (kJ)": "kJ",
        "Proteine (g)": "g (N x 6,25)",
        "Colesterolo (mg)": "mg",
        "Carboidrati disponibili (g)": "g",
        "Amido (g)": "g",
        "Zuccheri solubili (g)": "g",
        "Alcool (g)": "g",
        "Fibra totale (g)": "g"
    }

    results = {}

    for row in soup.find_all("tr", class_="corponutriente"):
        cells = row.find_all("td")
        if len(cells) >= 3:
            element_name = cells[0].get_text(strip=True)
            if element_name in elements_to_extract:
                element_value = cells[2].get_text(strip=True)
                unit = elements_to_extract[element_name]
                results[element_name] = f"{element_value} {unit}"

    return results

def main():
    workbook = openpyxl.Workbook()
    sheet = workbook.active
    sheet.title = "Dati Nutrizionali"
    sheet.append(["Link", "Nome Alimento", "Acqua (g)", "Energia (kcal)", "Energia (kJ)", "Proteine (g)", "Colesterolo (mg)", "Carboidrati disponibili (g)", "Amido (g)", "Zuccheri solubili (g)", "Alcool (g)", "Fibra totale (g)"])

    initial_html = get_html(BASE_URL)
    if initial_html:
        category_links = get_recipe_links(initial_html)

        for link in category_links:
            category_url = f"https://www.alimentinutrizione.it{link}"
            category_html = get_html(category_url)
            if category_html:
                food_data = extract_food_data(category_html)
                logging.info(f"Dati per {link}: {food_data}")

                if food_data:
                    row_data = [f"https://www.alimentinutrizione.it{link}"]
                    row_data.append(food_data.get("Nome Alimento", ""))
                    row_data.extend([food_data.get(key, "") for key in [
                        "Acqua (g)",
                        "Energia (kcal)",
                        "Energia (kJ)",
                        "Proteine (g)",
                        "Colesterolo (mg)",
                        "Carboidrati disponibili (g)",
                        "Amido (g)",
                        "Zuccheri solubili (g)",
                        "Alcool (g)",
                        "Fibra totale (g)"
                    ]])
                    sheet.append(row_data)

        workbook.save("dati_nutrizionali.xlsx")
        logging.info("Dati salvati in 'dati_nutrizionali.xlsx'")
    else:
        logging.error("Impossibile ottenere il contenuto HTML dalla pagina iniziale.")

if __name__ == "__main__":
    main()


# FINITO tranne porzione che deve essere sempre 100

In [26]:
import requests
from bs4 import BeautifulSoup
import openpyxl
import logging

logging.basicConfig(level=logging.INFO)

BASE_URL = "https://www.alimentinutrizione.it/tabelle-nutrizionali/ricerca-per-ordine-alfabetico"

def get_html(url):
    """
    Ottiene il contenuto HTML di una pagina.

    :param url: URL della pagina da cui ottenere l'HTML
    :return: Contenuto HTML della pagina, o None se si verifica un errore
    """
    try:
        response = requests.get(url)
        response.raise_for_status()  # Genera un'eccezione per errori HTTP
        return response.text
    except requests.exceptions.RequestException as e:
        logging.error(f"Errore nella richiesta a {url}: {e}")
        return None

def get_recipe_links(html):
    soup = BeautifulSoup(html, 'html.parser')
    recipe_links = []
    for link in soup.find_all('a', href=True):
        href = link.get('href')
        if href.startswith('/tabelle-nutrizionali/'):
            recipe_links.append(href)
    return recipe_links[:3]

def extract_food_data(html):
    """
    Estrae i dati nutrizionali e i dettagli di un alimento dalla pagina.

    :param html: Contenuto HTML della pagina
    :return: Tuple contenente gli headers, i dati nutrizionali e i dettagli dell'alimento
    """
    soup = BeautifulSoup(html, 'html.parser')
    food_data = []

    table = soup.find('table')
    headers = []
    if table:
        headers = [header.text.strip() for header in table.find_all('th')]
        rows = table.find_all('tr')
        for row in rows[1:]:
            cells = row.find_all('td')
            food_data.append([cell.text.strip() for cell in cells])
    else:
        logging.warning("Nessuna tabella trovata.")

    # Debug print per vedere il contenuto di food_data
    for idx, item in enumerate(food_data):
        print(f"Food Data {idx}: {item}")

    elements_to_extract = {
        "Acqua (g)": "g",
        "Energia (kcal)": "kcal",
        "Energia (kJ)": "kJ",
        "Proteine (g)": "g (N x 6,25)",
        "Colesterolo (mg)": "mg",
        "Carboidrati disponibili (g)": "g",
        "Amido (g)": "g",
        "Zuccheri solubili (g)": "g",
        "Alcool (g)": "g",
        "Fibra totale (g)": "g"
    }

    results = {}

    for row in soup.find_all("tr", class_="corponutriente"):
        cells = row.find_all("td")
        if len(cells) >= 3:
            element_name = cells[0].get_text(strip=True)
            if element_name in elements_to_extract:
                element_value = cells[2].get_text(strip=True)
                unit = elements_to_extract[element_name]
                results[element_name] = f"{element_value} {unit}"


    details = {
    "Nome Alimento": soup.find("h1", class_="article-title").get_text(strip=True) if soup.find("h1", class_="article-title") else "",
    "Categoria": next((item[1] for item in food_data if item[0] == "Categoria"), ""),
    "Codice Alimento": next((item[1] for item in food_data if item[0] == "Codice Alimento"), ""),
    "Parte Edibile": next((item[1] for item in food_data if item[0] == "Parte Edibile"), ""),
    "Porzione": next((item[1] for item in food_data if item[0] == "Porzione"), ""),
    "Nome Scientifico": next((item[1] for item in food_data if item[0] == "Nome Scientifico"), "")
    }

    return headers, food_data, results, details

def main():
    workbook = openpyxl.Workbook()
    sheet = workbook.active
    sheet.title = "Dati Nutrizionali"
    sheet.append(["Link", "Nome Alimento", "Categoria", "Codice Alimento", "Nome Scientifico", "Parte Edibile", "Porzione", "Acqua (g)", "Energia (kcal)", "Energia (kJ)", "Proteine (g)", "Colesterolo (mg)", "Carboidrati disponibili (g)", "Amido (g)", "Zuccheri solubili (g)", "Alcool (g)", "Fibra totale (g)"])

    initial_html = get_html(BASE_URL)
    if initial_html:
        category_links = get_recipe_links(initial_html)

        for link in category_links:
            category_url = f"https://www.alimentinutrizione.it{link}"
            category_html = get_html(category_url)
            if category_html:
                headers, food_data, results, details = extract_food_data(category_html)
                logging.info(f"Headers per {link}: {headers}")
                logging.info(f"Dati per {link}: {food_data}")
                logging.info(f"Dettagli per {link}: {details}")

                row_data = [
                    f"https://www.alimentinutrizione.it{link}",
                    details.get("Nome Alimento", ""),
                    details.get("Categoria", ""),
                    details.get("Codice Alimento", ""),
                    details.get("Nome Scientifico", ""),
                    details.get("Parte Edibile", ""),
                    details.get("Porzione", "")
                ]
                row_data.extend([results.get(key, "") for key in [
                    "Acqua (g)",
                    "Energia (kcal)",
                    "Energia (kJ)",
                    "Proteine (g)",
                    "Colesterolo (mg)",
                    "Carboidrati disponibili (g)",
                    "Amido (g)",
                    "Zuccheri solubili (g)",
                    "Alcool (g)",
                    "Fibra totale (g)"
                ]])
                sheet.append(row_data)

        workbook.save("dati_nutrizionali.xlsx")
        logging.info("Dati salvati in 'dati_nutrizionali.xlsx'")
    else:
        logging.error("Impossibile ottenere il contenuto HTML dalla pagina iniziale.")

if __name__ == "__main__":
    main()


Food Data 0: ['Categoria', 'Prodotti della pesca']
Food Data 1: ['Codice Alimento', '120010']
Food Data 2: ['Nome Scientifico', 'Engraulis enchrasicholus']
Food Data 3: ['English Name', 'Anchovy, fresh, raw']
Food Data 4: ['Parte Edibile', '75 %']
Food Data 5: ['Porzione', '150 g']
Details: {'Nome Alimento': 'Acciuga o alice', 'Categoria': 'Prodotti della pesca', 'Codice Alimento': '120010', 'Parte Edibile': '75 %', 'Porzione': '150 g', 'Nome Scientifico': 'Engraulis enchrasicholus'}
Food Data 0: ['Categoria', 'Prodotti della pesca']
Food Data 1: ['Codice Alimento', '120020']
Food Data 2: ['Nome Scientifico', 'Engraulis enchrasicholus']
Food Data 3: ['English Name', 'Anchovy, in oil']
Food Data 4: ['Parte Edibile', '100 %']
Food Data 5: ['Porzione', '50 g']
Details: {'Nome Alimento': "Acciuga o alice, sott'olio", 'Categoria': 'Prodotti della pesca', 'Codice Alimento': '120020', 'Parte Edibile': '100 %', 'Porzione': '50 g', 'Nome Scientifico': 'Engraulis enchrasicholus'}
Food Data 0: ['

# Codice ottimizzato:

In [2]:
!pip install aiohttp beautifulsoup4 openpyxl cachetools nest_asyncio

In [12]:
# Importazione delle librerie necessarie
import aiohttp  # Per le richieste HTTP asincrone
import asyncio  # Per la programmazione asincrona
from bs4 import BeautifulSoup  # Per il parsing HTML
import openpyxl  # Per la creazione di file Excel
import logging  # Per il logging delle informazioni
from cachetools import TTLCache  # Per la memorizzazione temporanea dei dati
import time  # Per misurare il tempo di esecuzione
import nest_asyncio  # Per permettere l'esecuzione di loop asyncio annidati

# Applica nest_asyncio per permettere l'esecuzione di loop annidati in ambienti come Jupyter Notebook
nest_asyncio.apply()

# Configurazione del logging per visualizzare le informazioni durante l'esecuzione
logging.basicConfig(level=logging.INFO)

# URL di base del sito web da cui estrarre i dati
BASE_URL = "https://www.alimentinutrizione.it/tabelle-nutrizionali/ricerca-per-ordine-alfabetico"

# Creazione di una cache per memorizzare le pagine HTML scaricate (durata: 1 ora)
html_cache = TTLCache(maxsize=100, ttl=3600)

async def get_html(session, url):
    """
    Funzione asincrona per ottenere il contenuto HTML di una pagina web.
    Usa una cache per evitare di scaricare più volte la stessa pagina.
    """
    if url in html_cache:
        return html_cache[url]  # Restituisce la versione in cache se disponibile

    try:
        async with session.get(url) as response:
            response.raise_for_status()  # Solleva un'eccezione per errori HTTP
            html = await response.text()  # Ottiene il contenuto HTML
            html_cache[url] = html  # Memorizza il contenuto nella cache
            return html
    except aiohttp.ClientError as e:
        logging.error(f"Errore nella richiesta a {url}: {e}")
        return None

def get_recipe_links(html):
    """
    Estrae i link delle ricette dalla pagina HTML principale.
    """
    soup = BeautifulSoup(html, 'html.parser')
    recipe_links = [link['href'] for link in soup.select('a[href^="/tabelle-nutrizionali/"]')]
    return recipe_links[:3]  # Restituisce solo i primi 3 link per limitare l'esecuzione

def extract_food_data(html):
    """
    Estrae i dati nutrizionali e i dettagli dell'alimento dalla pagina HTML.
    """
    soup = BeautifulSoup(html, 'html.parser')

    # Estrazione dei dettagli dell'alimento
    details = {
        "Nome Alimento": soup.select_one("h1.article-title").text.strip() if soup.select_one("h1.article-title") else "",
        "Categoria": soup.select_one('td:-soup-contains("Categoria") + td').text.strip() if soup.select_one('td:-soup-contains("Categoria") + td') else "",
        "Codice Alimento": soup.select_one('td:-soup-contains("Codice Alimento") + td').text.strip() if soup.select_one('td:-soup-contains("Codice Alimento") + td') else "",
        "Parte Edibile": soup.select_one('td:-soup-contains("Parte Edibile") + td').text.strip() if soup.select_one('td:-soup-contains("Parte Edibile") + td') else "",
        "Porzione": soup.select_one('td:-soup-contains("Porzione") + td').text.strip() if soup.select_one('td:-soup-contains("Porzione") + td') else "",
        "Nome Scientifico": soup.select_one('td:-soup-contains("Nome Scientifico") + td').text.strip() if soup.select_one('td:-soup-contains("Nome Scientifico") + td') else ""
    }

    # Lista degli elementi nutrizionali da estrarre
    elements_to_extract = [
        "Acqua (g)", "Energia (kcal)", "Energia (kJ)", "Proteine (g)", "Colesterolo (mg)",
        "Carboidrati disponibili (g)", "Amido (g)", "Zuccheri solubili (g)", "Alcool (g)", "Fibra totale (g)"
    ]

    # Estrazione dei valori nutrizionali
    results = {}
    for element in elements_to_extract:
        value_cell = soup.select_one(f'td:-soup-contains("{element}") ~ td:nth-of-type(3)')
        if value_cell:
            results[element] = value_cell.text.strip()
        else:
            results[element] = ""

    return details, results

async def process_category(session, link):
    """
    Funzione asincrona per processare una singola categoria di alimenti.
    """
    category_url = f"https://www.alimentinutrizione.it{link}"
    category_html = await get_html(session, category_url)
    if category_html:
        details, results = extract_food_data(category_html)
        logging.info(f"Dati estratti per {link}")
        return link, details, results
    return None

async def main():
    """
    Funzione principale asincrona che coordina l'estrazione dei dati e la creazione del file Excel.
    """
    workbook = openpyxl.Workbook()
    sheet = workbook.active
    sheet.title = "Dati Nutrizionali"

    # Aggiunge l'intestazione al foglio Excel
    sheet.append(["Link", "Nome Alimento", "Categoria", "Codice Alimento", "Nome Scientifico", "Parte Edibile", "Porzione",
                  "Acqua (g)", "Energia (kcal)", "Energia (kJ)", "Proteine (g)", "Colesterolo (mg)",
                  "Carboidrati disponibili (g)", "Amido (g)", "Zuccheri solubili (g)", "Alcool (g)", "Fibra totale (g)"])

    async with aiohttp.ClientSession() as session:
        initial_html = await get_html(session, BASE_URL)
        if initial_html:
            category_links = get_recipe_links(initial_html)

            # Crea e esegue le attività asincrone per ogni categoria
            tasks = [process_category(session, link) for link in category_links]
            results = await asyncio.gather(*tasks)

            # Elabora i risultati e li aggiunge al foglio Excel
            for result in results:
                if result:
                    link, details, nutrients = result
                    row_data = [
                        f"https://www.alimentinutrizione.it{link}",
                        details.get("Nome Alimento", ""),
                        details.get("Categoria", ""),
                        details.get("Codice Alimento", ""),
                        details.get("Nome Scientifico", ""),
                        details.get("Parte Edibile", ""),
                        details.get("Porzione", "")
                    ]
                    row_data.extend([nutrients.get(key, "") for key in [
                        "Acqua (g)", "Energia (kcal)", "Energia (kJ)", "Proteine (g)", "Colesterolo (mg)",
                        "Carboidrati disponibili (g)", "Amido (g)", "Zuccheri solubili (g)", "Alcool (g)", "Fibra totale (g)"
                    ]])
                    sheet.append(row_data)

    # Salva il file Excel
    workbook.save("dati_nutrizionali.xlsx")
    logging.info("Dati salvati in 'dati_nutrizionali.xlsx'")

async def run_main():
    """
    Funzione wrapper per eseguire main() e misurare il tempo di esecuzione.
    """
    start_time = time.time()
    await main()
    end_time = time.time()
    logging.info(f"Tempo di esecuzione totale: {end_time - start_time:.2f} secondi")

def execute_async():
    """
    Funzione per eseguire il codice asincrono in un ambiente sincrono (come Jupyter Notebook).
    """
    loop = asyncio.get_event_loop()
    loop.run_until_complete(run_main())

# Istruzioni per l'esecuzione
print("Per eseguire lo script, usa: execute_async()")

Per eseguire lo script, usa: execute_async()


In [1]:
'''stesso codice con Header con User-Agent per imitare un browser reale + sleep tra chiamate '''

import aiohttp
import asyncio
from bs4 import BeautifulSoup
import openpyxl
import logging
from cachetools import TTLCache
import time
import nest_asyncio
import random

# Applica nest_asyncio per permettere l'esecuzione di loop asyncio annidati
nest_asyncio.apply()

# Configurazione del logging
logging.basicConfig(level=logging.INFO)

# URL di base del sito web da cui estrarre i dati
BASE_URL = "https://www.alimentinutrizione.it/tabelle-nutrizionali/ricerca-per-ordine-alfabetico"

# Cache per memorizzare le pagine HTML scaricate (durata: 1 ora)
html_cache = TTLCache(maxsize=100, ttl=3600)

# Header con User-Agent per imitare un browser reale
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/126.0.0.0 Safari/537.36"
}

async def get_html(session, url):
    """
    Funzione asincrona per ottenere il contenuto HTML di una pagina web.
    Usa una cache per evitare di scaricare più volte la stessa pagina.
    Include un delay casuale e un User-Agent nell'header.
    """
    if url in html_cache:
        return html_cache[url]

    # Aggiunge un delay casuale tra 0 e 1 secondi
    await asyncio.sleep(random.uniform(0, 1))

    try:
        async with session.get(url, headers=HEADERS) as response:
            response.raise_for_status()
            html = await response.text()
            html_cache[url] = html
            return html
    except aiohttp.ClientError as e:
        logging.error(f"Errore nella richiesta a {url}: {e}")
        return None

def get_recipe_links(html):
    """
    Estrae i link delle ricette dalla pagina HTML principale.
    """
    soup = BeautifulSoup(html, 'html.parser')
    recipe_links = [link['href'] for link in soup.select('a[href^="/tabelle-nutrizionali/"]')]
    return recipe_links

def extract_food_data(html):
    """
    Estrae i dati nutrizionali e i dettagli dell'alimento dalla pagina HTML.
    """
    soup = BeautifulSoup(html, 'html.parser')

    details = {
        "Nome Alimento": soup.select_one("h1.article-title").text.strip() if soup.select_one("h1.article-title") else "",
        "Categoria": soup.select_one('td:-soup-contains("Categoria") + td').text.strip() if soup.select_one('td:-soup-contains("Categoria") + td') else "",
        "Codice Alimento": soup.select_one('td:-soup-contains("Codice Alimento") + td').text.strip() if soup.select_one('td:-soup-contains("Codice Alimento") + td') else "",
        "Parte Edibile": soup.select_one('td:-soup-contains("Parte Edibile") + td').text.strip() if soup.select_one('td:-soup-contains("Parte Edibile") + td') else "",
        "Porzione": soup.select_one('td:-soup-contains("Porzione") + td').text.strip() if soup.select_one('td:-soup-contains("Porzione") + td') else "",
        "Nome Scientifico": soup.select_one('td:-soup-contains("Nome Scientifico") + td').text.strip() if soup.select_one('td:-soup-contains("Nome Scientifico") + td') else ""
    }

    elements_to_extract = [
        "Acqua (g)", "Energia (kcal)", "Energia (kJ)", "Proteine (g)", "Colesterolo (mg)",
        "Carboidrati disponibili (g)", "Amido (g)", "Zuccheri solubili (g)", "Alcool (g)", "Fibra totale (g)"
    ]

    results = {}
    for element in elements_to_extract:
        value_cell = soup.select_one(f'td:-soup-contains("{element}") ~ td:nth-of-type(3)')
        if value_cell:
            results[element] = value_cell.text.strip()
        else:
            results[element] = ""

    return details, results

async def process_category(session, link):
    """
    Funzione asincrona per processare una singola categoria di alimenti.
    """
    category_url = f"https://www.alimentinutrizione.it{link}"
    category_html = await get_html(session, category_url)
    if category_html:
        details, results = extract_food_data(category_html)
        logging.info(f"Dati estratti per {link}")
        return link, details, results
    return None

async def main():
    """
    Funzione principale asincrona che coordina l'estrazione dei dati e la creazione del file Excel.
    """
    workbook = openpyxl.Workbook()
    sheet = workbook.active
    sheet.title = "Dati Nutrizionali"

    sheet.append(["Link", "Nome Alimento", "Categoria", "Codice Alimento", "Nome Scientifico", "Parte Edibile", "Porzione",
                  "Acqua (g)", "Energia (kcal)", "Energia (kJ)", "Proteine (g)", "Colesterolo (mg)",
                  "Carboidrati disponibili (g)", "Amido (g)", "Zuccheri solubili (g)", "Alcool (g)", "Fibra totale (g)"])

    async with aiohttp.ClientSession() as session:
        initial_html = await get_html(session, BASE_URL)
        if initial_html:
            category_links = get_recipe_links(initial_html)

            for link in category_links:
                result = await process_category(session, link)
                if result:
                    link, details, nutrients = result
                    row_data = [
                        f"https://www.alimentinutrizione.it{link}",
                        details.get("Nome Alimento", ""),
                        details.get("Categoria", ""),
                        details.get("Codice Alimento", ""),
                        details.get("Nome Scientifico", ""),
                        details.get("Parte Edibile", ""),
                        details.get("Porzione", "")
                    ]
                    row_data.extend([nutrients.get(key, "") for key in [
                        "Acqua (g)", "Energia (kcal)", "Energia (kJ)", "Proteine (g)", "Colesterolo (mg)",
                        "Carboidrati disponibili (g)", "Amido (g)", "Zuccheri solubili (g)", "Alcool (g)", "Fibra totale (g)"
                    ]])
                    sheet.append(row_data)

    workbook.save("dati_nutrizionali.xlsx")
    logging.info("Dati salvati in 'dati_nutrizionali.xlsx'")

async def run_main():
    """
    Funzione wrapper per eseguire main() e misurare il tempo di esecuzione.
    """
    start_time = time.time()
    await main()
    end_time = time.time()
    logging.info(f"Tempo di esecuzione totale: {end_time - start_time:.2f} secondi")

def execute_async():
    """
    Funzione per eseguire il codice asincrono in un ambiente sincrono (come Jupyter Notebook).
    """
    loop = asyncio.get_event_loop()
    loop.run_until_complete(run_main())

# Istruzioni per l'esecuzione
print("Per eseguire lo script, usa: execute_async()")

Per eseguire lo script, usa: execute_async()


In [2]:
execute_async()